In [48]:
import ray
import json
import numpy as np
import ray.rllib.algorithms.apex_dqn as apex_dqn
import ray.rllib.algorithms.dqn as dqn
from timeit import default_timer as timer


In [15]:

from logging import RootLogger


ray.init(ignore_reinit_error=True)


2022-08-24 15:28:29,611	INFO worker.py:1351 -- Calling ray.init() again after it has already been called.


Python version:,3.9.12
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [21]:
ENV = "BreakoutNoFrameskip-v4"
VERSION = "dqn_v1"
TRAINER = dqn

checkpoint_root = f'.checkpoints/{ENV}/{VERSION}'

config=TRAINER.DEFAULT_CONFIG.copy()

print(checkpoint_root)

.checkpoints/BreakoutNoFrameskip-v4/dqn_v1


In [22]:

config=dqn.DEFAULT_CONFIG.copy()
config['env'] = ENV
config["num_gpus"]=1
config["num_workers"]=8
config["num_cpus_per_worker"]=0.25
config['target_network_update_freq'] = 8000
config['replay_buffer_config']['capacity']= 10000

print(config['num_workers'])

8


In [23]:

algorithm = TRAINER.DQN(config=config)

2022-08-24 15:29:41,527	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
(pid=7494) /home/sam/micromamba/envs/ray/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=7494)   import imp
(pid=7495) /home/sam/micromamba/envs/ray/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=7495)   import imp
(pid=7500) /home/sam/micromamba/envs/ray/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=7500)   import imp
(pid=7498) /home/sam/micromamba/envs/ray/lib/pytho

In [24]:
algorithm._logdir = checkpoint_root
algorithm.logdir

'.checkpoints/BreakoutNoFrameskip-v4/dqn_v1/'

In [131]:
def cleanValue(value):
  valueType = type(value)
  if (valueType == dict):
    value = cleanDict(value)
  elif(valueType == list):
    value = cleanList(value)
  elif(valueType) == float:
    value == float(round(value,5))
  elif (valueType in [str, int, bool, type(None)]):
    pass
  elif(valueType in [np.float32, np.float64]):
    value = float(round(value,5))
  elif(valueType == np.ndarray):
    value = cleanList(value)
  elif(valueType == type(TRAINER.DEFAULT_CONFIG)):
    value == cleanDict(value)
  else:
    value = str(value)
  return value
  
def cleanList(dirtyList):
  cleanList=[]
  for value in dirtyList:
      cleanList.append(cleanValue(value))
  return cleanList

def cleanDict(dictionary):
  for key, value in zip(dictionary.keys(),dictionary.values()):
    dictionary[key] = cleanValue(value)
  return dictionary


In [134]:
for i in range(1000):
  result = algorithm.train()
  if i % 100 == 0:
    checkpoint = algorithm.save(checkpoint_root)
    with open(f'{checkpoint}/result.json', 'w') as fp:
        # fp.write(str(result))
        json.dump(cleanDict(result), fp,  indent=4)

